## Homework: Agents 
In this homework, we will learn more about function calling, and we will also explore MCP - model-context protocol.

### Preparation
First, we'll define a function that we will use when building our agent.
It will generate fake weather data:

In [ ]:
import random
known_weather_data = {
    'berlin': 20.0
}
def get_weather(city: str) -> float:
    city = city.strip().lower()
    if city in known_weather_data:
        return known_weather_data[city]
    return round(random.uniform(-5, 35), 1)

### Q1. Define function description
We want to use it as a tool for our agent, so we need to describe it
How should the description for this function look like? Fill in missing parts
```python
get_weather_tool = {
    "type": "function",
    "name": "<TODO1>",
    "description": "<TODO2>",
    "parameters": {
        "type": "object",
        "properties": {
            "<TODO3>": {
                "type": "string",
                "description": "<TODO4>"
            }
        },
        "required": [TODO5],
        "additionalProperties": False
    }
}
```
What did you put in TODO3?

#### Testing it (Optional)
If you have OpenAI API Key (or alternative provider), let's test it.
A question could be "What's the weather like in Germany?"
Experiment with different system prompts to have better answers from the system.
You can use chat_assistant.py or implement everything yourself

`wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py`

In [ ]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather", # The name of the function as a string
    "description": 
        "Get the current temperature for a given city.", # What the function does
    "parameters": {
        "type": "object",
        "properties": {
            "city": { # The parameter expected by the function
                "type": "string",
                "description": 
                    "The name of the city to get the weather for." # A parameter description
            }
        },
        "required": ["city"], # The required parameter as a list
        "additionalProperties": False
    }
}

In [ ]:
print(get_weather_tool)

{'type': 'function', 'name': 'get_weather', 'description': 'Get the current temperature for a given city.', 'parameters': {'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'The name of the city to get the weather for.'}}, 'required': ['city'], 'additionalProperties': False}}


In [ ]:
# 1. Register the Tool for this session:
import chat_assistant

tools = chat_assistant.Tools() 
# chat_assistant will automatically pass it to OpenAI API
tools.add_tool(get_weather, get_weather_tool)

In [ ]:
# 2. Set Up the Chat: 

# Create a developer/system prompt and chat messages
developer_prompt = """
You are a helpful weather assistant. Use the get_weather tool to answer weather questions.
""".strip()

chat_messages = [
    {"role": "developer", "content": developer_prompt},
    {"role": "user", "content": "What's the weather like in Denmark?"}
]

In [ ]:
from openai import OpenAI
client = OpenAI()

In [ ]:
# 3a. Use chat_assistant.py
chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_assistant.ChatInterface(),
    client=client
)
chat.run()

Chat ended.


In [ ]:
# 3b. Call the OpenAI API (skip if using chat_assistant.py) 

response = client.responses.create( # Use the new responses API
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools.get_tools() # Use OpenAI API directly (without chat_assistant)
)

In [ ]:
# 4. Handle Function Calls (skip if using chat_assistant.py) 
for entry in response.output:
    # If the model decides to call your function, process the function call
    if entry.type == 'function_call':
        # Call your function with the arguments
        import json
        arguments = json.loads(entry.arguments)
        result = get_weather(**arguments)
        print("Function call result:", result)
        
        # Append the function call output as a string
        chat_messages.append({
            "type": "function_call_output",
            "call_id": entry.call_id,
            "output": str(result), 
        })
    elif entry.type == 'message':
        print(entry.content[0].text)

Function call result: 32.4


5. Experiment with Prompts
Try changing developer_prompt to see how the model's behavior changes, e.g.:

* "You are a weather expert. Always use the get_weather tool for city weather questions."
* "You can answer general questions, but use the tool for specific cities."


### Answer:
* Q: What did you put in TODO3?
* A: city

### Q2. Adding another tool
Let's add another tool - a function that can add weather data to our database:

In [ ]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

Now let's write a description for it.
What did you write?
Optionally, you can test it after adding this function.

In [ ]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": 
        "Set or update the temperature for a given city in the weather database.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": 
                    "The name of the city to set the temperature for."
            },
            "temp": {
                "type": "number",
                "description": 
                    "The temperature value to set for the city."
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

### Answer:
* Q: What did you write?
* A: "Set or update the temperature for a given city in the weather database."

#### MCP
MCP stands for Model-Context Protocol. It allows LLMs communicate with different tools (like Qdrant). It's function calling, but one step further:
* A tool can export a list of functions it has
* When we include the tool to our Agent, we just need to include the link to the MCP server

### Q3. Install FastMCP
Let's install a library for MCP - FastMCP:
`pip install fastmcp`
What's the version of FastMCP you installed?

In [ ]:
%pip install fastmcp

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.7/630.7 kB 26.1 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.9.0-py3-none-any.whl size=11103 sha256=098c20cca5eba5f2b872972de3ad0cdb8493847a989a0a0a4905af6c745a28c8
  Stored in directory: /home/codespace/.cache/pip/wheels/cc/ae/36/ee17d1de094fcb61e24106cb329b5103861e819f94bef5e10a
Successfully built pyperclip
Note: you may need to restart the kernel to use updated packages.


### Answer:
* Q: What's the version of FastMCP you installed?
* A: fastmcp-2.11.2

### Q4. Simple MCP Server
A simple MCP server from the documentation looks like that:

In [ ]:
# weather_server.py
from fastmcp import FastMCP
mcp = FastMCP("Demo 🚀")
@mcp.tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b
if __name__ == "__main__":
    mcp.run()

In our case, we need to write docstrings for our functions.
Let's ask ChatGPT for help:

In [ ]:
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.
    Parameters:
        city (str): The name of the city for which to retrieve weather data.
    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()
    if city in known_weather_data:
        return known_weather_data[city]
    return round(random.uniform(-5, 35), 1)

def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.
    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.
    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

Let's change the example for our case and run it
What do you see in the output?
Look for a string that matches this template:
Starting MCP server 'Demo 🚀' with transport `<TODO>`
What do you have instead of `<TODO>`?

1. Write MCP server (See weather_server.py Python script)
2. Run in terminal:
    `python weather_server.py`
3. Observe the output

### Answer:
* Q: What do you see in the output? What do you have instead of <TODO>?
* A: stdio

### Q5. Protocol
There are different ways to communicate with an MCP server. Ours is currently running using standart input/output, which means that the client write something to stdin and read the answer using stdout.
Our weather server is currently running.
This is how we start communitcating with it:

* First, we send an initialization request -- this way, we register our client with the server:

```json
{"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": true}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}
```

We should get back something like that, which is an aknowledgement of the request:
```json
{"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":true}},"serverInfo":{"name":"Demo 🚀","version":"1.9.4"}}}
```
* Next, we reply back, confirming the initialization:

```json
{"jsonrpc": "2.0", "method": "notifications/initialized"}
```

We don't expect to get anything in response

* Now we can ask for a list of available methods:

```json
{"jsonrpc": "2.0", "id": 2, "method": "tools/list"}
```

* Let's ask the temperature in Berlin:

```json
{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "<TODO>", "arguments": {<TODO>}}}
```

* What did you get in response?

Run in terminal after preliminary JSON protocol:

{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city": "berlin"}}}

### Answer:
* Q: What did you get in response?
* A: 6.9

### Q6. Client
We typically don't interact with the server by copy-pasting commands in the terminal.
In practice, we use an MCP Client. Let's implement it.
FastMCP also supports MCP clients:
```python
from fastmcp import Client
async def main():
    async with Client(<TODO>) as mcp_client:
        # TODO
```
Use the client to get the list of available tools of our script. How does the result look like?
If you're running this code in Jupyter, you need to pass an instance of MCP server to the Client:
```python
import weather_server
async def main():
    async with Client(weather_server.mcp) as mcp_client:
        # ....
```
If you run it in a script, you will need to use asyncio:
```python
import asyncio
async def main():
    async with Client("weather_server.py") as mcp_client:
        # ...
```
```python
if __name__ == "__main__":
    test = asyncio.run(main())
```
Copy the output with the available tools when filling in the homework form.



Write mcp_client.py script to:

1. Start the MCP server (if not already running).
2. Open a subprocess to communicate with it via stdin/stdout.
3. Send the JSON-RPC requests (python mcp_client.py).
4. Read and parse the responses.

### Answer:
* Q: Copy the output with the available tools when filling in the homework form
* A: {'name': 'get_weather', 'description': 'Retrieves the temperature for a specified city.\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\nReturns:\n    float: The temperature associated with the city.', 'inputSchema': {'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, 'outputSchema': {'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, '_meta': {'_fastmcp': {'tags': []}}}
{'name': 'set_weather', 'description': "Sets the temperature for a specified city.\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", 'inputSchema': {'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'Temp', 'type': 'number'}}, 'required': ['city', 'temp'], 'type': 'object'}, 'outputSchema': {'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, '_meta': {'_fastmcp': {'tags': []}}}